In [ ]:
from ipywidgets import widgets,interactive
import numpy as np
from time import time, sleep
from h5py import File as h5py_File
import matplotlib.pyplot as plt
from zipfile import ZipFile
from os.path import abspath , exists
from os import remove, mkdir, system
from datetime import datetime

In [ ]:
width_png=529
height_png=400

In [ ]:
image_height=300
image_width=int(width_png*image_height/height_png)
raw_bytes_width=100
tests_results_width=500
tmp_image_name='tmp_ex1.svg'

In [ ]:
s_data_file_name='data_for_ex1'
s_data_dir_name='data'
nb_subzip_files=10

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    s_data_dir_name='data_colab'
except:
    IN_COLAB = False
    s_data_dir_name='data'
if IN_COLAB:
    if exists(s_data_dir_name)==False:
        mkdir(s_data_dir_name)
    for i in range(nb_subzip_files):
        s_cmd='wget https://github.com/patrickhaddadteaching/TRNG_ex1/raw/main/data/data_for_ex1.zip%d'%i
        system(s_cmd)
        system('mv data_for_ex1.zip%d ./%s/'%(i,s_data_dir_name))

In [ ]:
path_to_zip_file='%s_tmp.zip'%(s_data_file_name)
f_zip=open(path_to_zip_file, 'w')
for i in range(nb_subzip_files):
    s_subzip_path='%s/%s.zip%d'%(s_data_dir_name,s_data_file_name,i)
    if exists(s_subzip_path):
        #print('%s EXIST'%s_subzip_path)
        f_subzip=open(s_subzip_path, 'r')
        subzip_bytes=np.fromfile(f_subzip, dtype=np.uint8)
        f_subzip.close()
        subzip_bytes.tofile(f_zip)
    else:
        print('%s DOES NOT EXIST'%s_subzip_path)
f_zip.close()

In [ ]:
zip_ref=ZipFile(path_to_zip_file, 'r')
zip_ref.extractall('.')

In [ ]:
hdf5_file = '%s.h5'%s_data_file_name
data_file = h5py_File(hdf5_file,'r')
v_ratio_period_sample_for_interact=np.array(data_file['parameters/v_ratio_period_sample'][0,:],dtype=np.uint32)
v_jitter_for_interact=data_file['parameters/v_jitter'][0,:]
v_duty_cycle_interact=data_file['parameters/v_duty_cycle'][0,:]
v_raw_rng_image=data_file['images/rng_image'][0,:]
m_raw_bits_from_one_RO=data_file['raw_bits/m_raw_bits_from_one_RO'][:]
data_file.close()
#remove('%s'%hdf5_file)
nb_test_per_corner=m_raw_bits_from_one_RO.shape[3]
nb_bits=m_raw_bits_from_one_RO.shape[4]

In [ ]:
if exists('random_values')==False:
    mkdir('random_values')

In [ ]:

raw_bytes=widgets.Textarea(value='',rows=5000,placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%(image_width/4)))
html_download=widgets.HTML(value='',layout=widgets.Layout(width='%dpx'%(image_height*1.25+tests_results_width)))
rng_image=widgets.Image(value=v_raw_rng_image.tobytes(), format='png',layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%image_width))

v_x=np.array([ 0,  1,  2,  4,  8,  3,  5,  6,  9, 10, 12,  7, 11, 13, 14, 15], dtype=np.uint8)
v_s_x=['0000', '0001', '0010', '0100', '1000', '0011', '0101', '0110', '1001', '1010', '1100', '0111', '1011', '1101', '1110', '1111']
nb_tau=100 
def generate_rnd(chosen_sample_period,chosen_jitter,chosen_duty_cycle,save_data_in_file):


    index_sample_period=np.where(chosen_sample_period==v_ratio_period_sample_for_interact)[0][0]
    index_chosen_jitter=v_std_dev_value_widgets_option.index(chosen_jitter)
    index_chosen_chosen_duty_cycle=duty_cycle_value_widgets_option.index(chosen_duty_cycle)
    
    raw_bytes.value=''
    
    tmp_rand=np.random.randint(0,nb_test_per_corner,dtype=np.uint32)
    v_raw_bits=m_raw_bits_from_one_RO[index_chosen_chosen_duty_cycle,index_chosen_jitter,index_sample_period,tmp_rand,:]
        
    nb_nibble=int((nb_bits)/4)
    v_nibble=np.zeros((nb_nibble,),dtype=np.uint8)
    for i in range(4):
        v_nibble+=np.array(v_raw_bits[i:4*nb_nibble:4],dtype=np.uint8)*(2**i)

    v_hist=np.zeros((16,),dtype=np.uint32)
    for i in range(16):
        v_hist[i]=((v_nibble==v_x[i]).sum())
        

    nb_words_to_be_used=v_nibble.shape[0]-nb_tau
    v_T5=np.zeros((nb_tau,),dtype=np.float64)
    for tau in range(nb_tau):
        v_T5[tau]=(((v_nibble[:nb_words_to_be_used])^(v_nibble[tau+1:tau+nb_words_to_be_used+1])).mean())-7.5
    
    fig_height=image_height/80
    fig, ax = plt.subplots(2, 1,figsize=[fig_height*1.5,fig_height])
    ax[0].bar(np.arange(16),v_hist)
    ax[0].set_title('4-bits words distribution')
    ax[0].set_xlabel('4-bits word value')
    ax[0].set_xticks(np.arange(16))
    ax[0].set_xticklabels(v_s_x,rotation='vertical')
    
    ax[0].plot(np.arange(17)-0.5,np.ones(17)*370,'r')
    ax[0].plot(np.arange(17)-0.5,np.ones(17)*254,'r')
    
    
    ax[1].plot(v_T5)
    ax[1].set_xlabel('Lag')
    ax[1].set_title('4-bits words Autocorrelation')
    ax[1].plot(np.arange(nb_tau),np.ones(nb_tau)*0.23,'r')
    ax[1].plot(np.arange(nb_tau),np.ones(nb_tau)*(-0.23),'r')
        
    fig.tight_layout(pad=0, h_pad=2.0, w_pad=None)
    plt.show()
    s_line=''
    for i in range(nb_nibble):
        s_line='%s%X'%(s_line,v_nibble[i])
    raw_bytes.value=s_line   
    
    if save_data_in_file==True:
        
        now = datetime.now()
        str_time = now.strftime("%d-%m-%Y_%Hh%Mm%Ss%fms")
        randomvalues_file_name='ex1_a%s_jit%s_D%d_%s.txt'%(chosen_duty_cycle,chosen_jitter,chosen_sample_period,str_time)
        fid_rnd_file=open('random_values/%s'%randomvalues_file_name,'w')
        fid_rnd_file.write(s_line)
        fid_rnd_file.close()

        path_to_download='%s'%(randomvalues_file_name)
        s_line_for_download='<p> random numbers are saved in the following file %s stored in %s</p>'%(path_to_download,abspath('random_values'))
        html_download.value=s_line_for_download
    else:
        s_line_for_download=''
        html_download.value=s_line_for_download

v_std_dev_value_widgets_option=[]
for i in range(v_jitter_for_interact.shape[0]):
    v_std_dev_value_widgets_option.append('%.2e'%(v_jitter_for_interact[i]))
std_dev_value_widgets=widgets.SelectionSlider(description=r'\(\frac{\sigma_{tot}}{T_{1}}\)',options=v_std_dev_value_widgets_option,layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))

duty_cycle_value_widgets_option=[]
for i in range(v_duty_cycle_interact.shape[0]):
    duty_cycle_value_widgets_option.append('%.2f'%(v_duty_cycle_interact[i]))
    
duty_cycle_value_widgets=widgets.SelectionSlider(description=r'\(\alpha_{1}\)',options=duty_cycle_value_widgets_option,layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))

sample_period_value_widgets=widgets.SelectionSlider(description=r'\(D\)',options=list(v_ratio_period_sample_for_interact),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)),readout_format='.1f')
save_data_in_file_widgets=widgets.Checkbox(value=False,description='SAVE RANDOM NUMBERS IN A FILE',disabled=False,button_style='', tooltip='Description',icon='check')

interactive_plot = interactive(generate_rnd,{'manual': True}, chosen_sample_period=sample_period_value_widgets,chosen_jitter =std_dev_value_widgets ,chosen_duty_cycle=duty_cycle_value_widgets,save_data_in_file=save_data_in_file_widgets)
interactive_plot.children[-2].description='GENERATE RANDOM VALUES'
interactive_plot.children[-2].layout=widgets.Layout(width='%dpx'%((int((raw_bytes.layout.width)[:-2])+tests_results_width)*0.8))
interactive_plot.children[-3].layout=widgets.Layout(width=interactive_plot.children[-2].layout.width)
v_box_top=widgets.VBox([widgets.VBox([widgets.HBox([rng_image,raw_bytes,interactive_plot.children[-1]]),widgets.VBox([widgets.HBox([widgets.VBox([interactive_plot.children[2],interactive_plot.children[1],interactive_plot.children[0]]),widgets.VBox([interactive_plot.children[4],interactive_plot.children[3]])])])]),html_download])
display(v_box_top)